# Simulation with default parameters and periodic boundary conditions on TKD mesh

The following notebook outlines a sample simulation of the perfusion and gas exchange model on a periodic mesh mimicking an acinus as a collection of pseudo-spherical alveoli. We consider an infinite packing of tetrakaidecahedra, each one of which represents a single, $200 \, \mu \text{m}$ alveolus. We start without considering hemoglobin effects.

Import necessary packages and add the working directory to the system path.

In [9]:
import sys
import os
import dolfin
from ipygany import Scene, PolyMesh, ColorBar, IsoColor
from ipywidgets import Play, IntProgress, link, VBox
sys.path.append(os.getcwd()[:-6])

Import the model modules.

In [10]:
from src.model import PerfusionGasExchangeModel
from src.params import params

Set a target folder for `vtk` files.

In [11]:
folder = "periodic-bcs-tkd"
path = os.path.join("../raw-data", folder)

Instance the model and run simulation.

In [12]:
model = PerfusionGasExchangeModel(folder_path=path, params=params)
model.import_mesh(
    os.path.join("../raw-data", "TKD_test.xml"), type="xml", periodic=True
)
model.sim_p(save=True, meshtype="tkd")
model.sim_bst(final_time=1, num_steps=15, save=True, hb=False)

Solving linear variational problem.
No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.558e+06 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 3.387e-09 (tol = 1.000e-10) r (rel) = 2.174e-15 (tol = 1.000e-09)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Finished time step 1/15 (7%)

No Jacobian form specified for nonlinear variational problem.
Differentiating residual form F to obtain Jacobian J = F'.
Solving nonlinear variational problem.
  Newton iteration 0: r (abs) = 1.016e+03 (tol = 1.000e-10) r (rel) = 1.000e+00 (tol = 1.000e-09)
  Newton iteration 1: r (abs) = 4.694e-10 (tol = 1.000e-10) r (rel) = 4.620e-13 (tol = 1.000e-09)
  Newton solver finished in 1 iterations and 1 linear solver iterations.
Finished time step 2/15 (13%)

No Jacobian form specified for nonlinear vari

In [14]:
boundaries = dolfin.File(model.folder_path+'/bnd/bnd.pvd')
boundaries << model.boundaries

Visualize using `iygany`. As an example, we start with $p$. Currently, prior knowledge of the `vtu` file is needed to visualize this. There's probably a better way to do this.

In [15]:
mesh = PolyMesh.from_vtk(os.path.join(model.folder_path, 'p/p000000.vtu'))
iso = IsoColor(mesh, input='f_98', min=8, max=12)
scene = Scene([iso])
bar = ColorBar(iso)
VBox(children=(scene, bar))

Next we visualize $p_{\text{O}_{2}}$.

In [19]:
mesh_dynamic = PolyMesh.from_vtk(
    os.path.join(model.folder_path, 'bst/pO2000000.vtu')
)

def load_step(change):
    mesh_dynamic.reload(
        os.path.join(model.folder_path, 'bst/pO20000{}.vtu').format(
            str(change['new']).zfill(2)
        ), reload_vertices=True
    )

play = Play(description='Step:', min=1, max=15, value=1)
play.observe(load_step, names=['value'])

progress = IntProgress(value=1, step=1, min=1, max=15)
link((progress, 'value'), (play, 'value'))

stepper = VBox((play, progress))
iso = IsoColor(mesh_dynamic, input='f_128-0', min=39.5, max=40)
bar = ColorBar(iso)
scene = Scene([iso])

VBox(children=(scene, stepper, bar))